# Evaluación de PC-SMOTE con Grid Search en el dataset Shuttle (Generación de caso base y datasets aumentados)


In [1]:
# lo que hace es modificar la lista de rutas de búsqueda de módulos de Python (sys.path) para incluir las carpetas ../scripts y ../datasets como ubicaciones adicionales donde Python puede buscar módulos o paquetes cuando hacés un import.
import sys
sys.path.append("../scripts")
sys.path.append("../datasets")

## Importación de módulos y librerías necesarias


In [2]:
# --- Módulos propios del proyecto ---
from cargar_dataset import cargar_dataset                      # Función para cargar datasets según configuración
from config_datasets import config_datasets                    # Diccionario de configuración de datasets
from evaluacion import evaluar_sampler_holdout                 # Evaluación de sobremuestreo con partición hold-out
from pc_smote import PCSMOTE                                   # Implementación principal de PCSMOTE
from graficador2d import Graficador2D                       # Clase para graficar resultados en 2D
from isolation_cleaner import IsolationCleaner                 # Clase para limpieza de outliers con Isolation Forest
from Utils import Utils                                       # Clase utilitaria con funciones auxiliares
from limpiador import LimpiadorOutliers                                 # Clase para limpieza de datos

from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN

# --- Librerías estándar de Python ---
from datetime import datetime, timedelta                       # Manejo de fechas y tiempos
from itertools import product                                  # Generación de combinaciones de parámetros
import os                                                      # Operaciones con el sistema de archivos
from pathlib import Path

import traceback
# --- Librerías científicas ---
import numpy as np                                              # Operaciones numéricas y algebra lineal
import pandas as pd                                             # Manipulación y análisis de datos tabulares
from scipy.stats import uniform                                 # Distribuciones para búsqueda de hiperparámetros

# --- Scikit-learn: preprocesamiento ---
from sklearn.preprocessing import LabelEncoder, StandardScaler # Codificación de etiquetas y escalado de datos
from sklearn.pipeline import make_pipeline, Pipeline            # Creación de pipelines de procesamiento y modelado
from sklearn.preprocessing import RobustScaler

# --- Scikit-learn: división y validación ---
from sklearn.model_selection import (
    train_test_split,                                           # División de datos en train/test
    StratifiedKFold,                                            # Validación cruzada estratificada
    RandomizedSearchCV                                          # Búsqueda aleatoria de hiperparámetros
)

# --- Scikit-learn: reducción de dimensionalidad ---
from sklearn.decomposition import PCA                           # Análisis de Componentes Principales

# --- Scikit-learn: métricas ---
from sklearn.metrics import (
    f1_score,                                                    # Métrica F1-Score
    balanced_accuracy_score,                                     # Precisión balanceada
    matthews_corrcoef,                                           # Coeficiente MCC
    cohen_kappa_score,                                           # Kappa de Cohen
    make_scorer                                            
)

# --- Scikit-learn: clasificadores ---
from sklearn.ensemble import RandomForestClassifier             # Clasificador Random Forest
from sklearn.linear_model import LogisticRegression             # Regresión logística
from sklearn.svm import SVC                                      # Máquinas de Vectores de Soporte (SVM)

from sklearn.exceptions import ConvergenceWarning
import warnings

RANDOM_STATE = 42
RUTA_CLASICOS: str = "../datasets/datasets_aumentados/resampler_clasicos/",

c:\Users\FamiliaNatelloMedina\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Generación del caso base

Este código realiza dos tareas principales para cada dataset configurado en `config_datasets`:

1. **Generar el caso base** (subcarpeta `datasets_aumentados/base/`):
   - Se crea un directorio específico para almacenar la versión original del dataset sin ningún tipo de sobremuestreo.
   - El dataset se carga utilizando la misma función `cargar_dataset` empleada en el pipeline principal.
   - Si las etiquetas (`y`) están en formato de texto u objeto, se convierten a valores numéricos con `LabelEncoder`.
   - Se realiza una división estratificada en conjuntos de entrenamiento y prueba (`train/test`) utilizando `train_test_split` con una proporción 70/30 y una semilla fija para asegurar reproducibilidad.
   - Se guardan dos archivos CSV: `<nombre_dataset>_train.csv` y `<nombre_dataset>_test.csv`.

In [3]:
def generar_caso_base(
    nombre_dataset: str,
    config: dict,
    ruta_base: str = "../datasets/datasets_aumentados/base/",
    test_size: float = 0.2,
    random_state: int = 42,
    overwrite: bool = False,
    porcentaje_limpieza: float = 0.0,
):
    """
    Genera el caso base (sin sobremuestreo) aplicando:
      1) Carga del dataset según config_datasets y cargar_dataset()
      2) Split train/test
      3) RobustScaler (fit SOLO en X_train, transform en X_train/X_test)
      4) (Opcional) Limpieza de outliers con IsolationCleaner (IsolationForest por percentil)
         SOLO sobre X_train_scaled.

    IMPORTANTE:
    - Aquí `porcentaje_limpieza` se usa para LIMPIAR realmente el conjunto
      de entrenamiento con IsolationCleaner (si > 0). Internamente se pasa como
      `percentil_umbral`.
    - El valor de `porcentaje_limpieza` se refleja en el nombre del archivo de TRAIN
      como sufijo `_I{porcentaje_limpieza}` para dejar traza.
    - El TEST nunca se limpia con IsolationForest, por eso su nombre no lleva sufijo `I*`.

    El resultado son dos CSV:
      - {ruta_base}/{nombre_dataset}_I{porcentaje_limpieza}_train.csv
        (train escalado y, si corresponde, limpiado con IF por percentil)
      - {ruta_base}/{nombre_dataset}_test.csv
        (test escalado, sin limpieza IF)

    Estos archivos se usan como base para:
      - el caso base (sin sobremuestreo),
      - las técnicas clásicas (SMOTE, BorderlineSMOTE, ADASYN),
      - y PC-SMOTE.
    """

    os.makedirs(ruta_base, exist_ok=True)

    path_train = os.path.join(
        ruta_base,
        f"{nombre_dataset}_I{porcentaje_limpieza}_train.csv"
    )
    path_test = os.path.join(
        ruta_base,
        f"{nombre_dataset}_test.csv"
    )

    if not overwrite and os.path.exists(path_train) and os.path.exists(path_test):
        print(f"⚠️ Caso base ya existe para {nombre_dataset}. Usando archivos existentes.")
        return path_train, path_test

    # 1) Cargar dataset crudo según config_datasets usando cargar_dataset()
    df_features, y, clases = cargar_dataset(
        path=config["path"],
        clase_minoria=config.get("clase_minoria"),
        col_features=config.get("col_features"),
        col_target=config.get("col_target"),
        sep=config.get("sep"),
        header=config.get("header"),
        binarizar=config.get("binarizar", False),
        tipo=config.get("tipo", "tabular"),
        impute=config.get("impute", "median"),
        na_values=config.get("na_values", ('?', 'NA', 'None')),
        dataset_name=config.get("dataset_name", nombre_dataset),
        names=config.get("esquema"),
    )

    col_target   = config["col_target"]
    col_features = config["col_features"]

    # X son directamente las features que devuelve cargar_dataset
    X = df_features  # ya tiene col_features como columnas
    # y es el vector que devolvió cargar_dataset (np.ndarray)
    # train_test_split acepta ndarray, no hace falta convertirlo, salvo para el concat final

    # 2) Train / test split (estratificado)
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=test_size,
        random_state=random_state,
        stratify=y,
    )

    print(f"[{nombre_dataset}] Split: X_train={X_train.shape}, X_test={X_test.shape}")

    # 3) Escalado robusto (fit en train, transform en train y test)
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled  = scaler.transform(X_test)

    # 4) (Opcional) Limpieza de outliers con IsolationCleaner SOLO sobre TRAIN
    if porcentaje_limpieza is not None and porcentaje_limpieza > 0:
        print(f"[{nombre_dataset}] Aplicando IsolationCleaner (percentil={porcentaje_limpieza}%) sobre TRAIN")

        X_train_scaled, y_train, info_limpieza = IsolationCleaner.limpiarOutliers(
            X=X_train_scaled,
            y=y_train,  # ya es np.ndarray; tu IsolationCleaner trabaja con np.asarray internamente
            percentil_umbral=float(porcentaje_limpieza),
            contamination="auto",
            n_estimators=200,
            max_samples="auto",
            random_state=random_state,
            bootstrap=False,
            normalizar_scores=False,
            devolver_info=True,
            verbose=True,
        )

        removed_total = info_limpieza.get("removed_total", 0)
        total_inicial = removed_total + len(y_train)
        print(
            f"[{nombre_dataset}] Limpieza IF (percentil): "
            f"removidos={removed_total} / total_inicial≈{total_inicial}"
        )

    # 5) Reconstruir DataFrames con columnas
    df_train = pd.concat(
        [
            pd.DataFrame(X_train_scaled, columns=col_features),
            pd.Series(y_train, name=col_target),
        ],
        axis=1,
    )

    df_test = pd.concat(
        [
            pd.DataFrame(X_test_scaled, columns=col_features),
            pd.Series(
                y_test,
                name=col_target,
            ),
        ],
        axis=1,
    )

    # 6) Guardar CSV base
    df_train.to_csv(path_train, index=False)
    df_test.to_csv(path_test, index=False)

    print(f"✅ Caso base generado para {nombre_dataset}")
    print(f"   Train: {path_train}  (rows={df_train.shape[0]})")
    print(f"   Test : {path_test}   (rows={df_test.shape[0]})")

    return path_train, path_test


In [4]:
def aumentar_dataset_pcsmote_y_guardar(
    nombre_dataset,
    X_train_base,
    y_train_base,
    percentil_densidad,
    percentil_riesgo,
    criterio_pureza,
    col_target="target",
    percentil_isolation_etiqueta:float = 0.0
):
    try:
        # 1) PCSMOTE sobre el TRAIN limpio (ya escalado)
        sampler = PCSMOTE(
            random_state=42,
            percentil_densidad=percentil_densidad,
            percentil_entropia=50 if criterio_pureza == "entropia" else None,
            percentil_riesgo=percentil_riesgo,
            criterio_pureza=criterio_pureza,
            factor_equilibrio=1,
            metric="euclidean",
        )

        if hasattr(sampler, "fit_resample_multiclass"):
            X_res, y_res = sampler.fit_resample_multiclass(X_train_base, y_train_base)
        else:
            X_res, y_res = sampler.fit_resample(X_train_base, y_train_base)

        # 2) Guardar TRAIN aumentado
        out_dir = Path("../datasets/datasets_aumentados/")
        out_dir.mkdir(parents=True, exist_ok=True)

        fname_train = (
            f"pcsmote_{nombre_dataset}_"
            f"D{percentil_densidad}_R{percentil_riesgo}_"
            f"{Utils.tag_p(criterio_pureza)}_I{percentil_isolation_etiqueta}_train.csv"
        )
        fname_train = Utils.safe_token(fname_train)
        p_train = out_dir / fname_train

        df_train = pd.DataFrame(X_res)
        df_train[col_target] = y_res

        df_train.to_csv(p_train, index=False)

        return str(p_train), sampler

    except Exception as e:
        traceback.print_exc()
        print(f"❌ Error al aumentar dataset {nombre_dataset}: {e}")
        return None, None


In [5]:
def generar_aumentaciones_clasicas_y_guardar(
    nombre_dataset: str,
    X_train: pd.DataFrame,
    y_train: pd.Series,
    col_target: str,
    ruta_clasicos: str = "../datasets/datasets_aumentados/resampler_clasicos/",
    overwrite: bool = False,
    percentil_isolation_etiqueta:float = 0.0
):
    """
    Genera datasets aumentados con técnicas clásicas a partir de X_train, y_train:
      - SMOTE
      - BorderlineSMOTE
      - ADASYN

    Guarda los resultados en CSV en ruta_clasicos/ con nombres:
      smote_{dataset}_train.csv
      borderlinesmote_{dataset}_train.csv
      adasyn_{dataset}_train.csv
    """
    os.makedirs(ruta_clasicos, exist_ok=True)

    columnas = list(X_train.columns)

    print(f"🔧 Aumentación clásica (en memoria) para: {nombre_dataset}")
    print(f"   X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

    resamplers = [
        ("smote", SMOTE(random_state=RANDOM_STATE)),
        ("borderlinesmote", BorderlineSMOTE(random_state=RANDOM_STATE, kind="borderline-1")),
        ("adasyn", ADASYN(random_state=RANDOM_STATE)),
    ]

    for nombre_resampler, resampler in resamplers:
        nombre_archivo = f"{nombre_resampler}_{nombre_dataset}_I{percentil_isolation_etiqueta}_train.csv"
        path_salida = os.path.join(ruta_clasicos, nombre_archivo)

        if not overwrite and os.path.exists(path_salida):
            print(f"   ⚪ Omitido ({nombre_resampler}), ya existe: {nombre_archivo}")
            continue

        print(f"   🔁 Aplicando {nombre_resampler} ...")

        try:
            # fit_resample trabaja con ndarrays
            X_res, y_res = resampler.fit_resample(X_train.values, y_train.values)
        except ValueError as e:
            # Caso típico: ADASYN (u otro) decide no generar muestras
            print(f"   ⚠️ {nombre_resampler} no generó muestras sintéticas: {e}")
            print(f"      → Se omite guardar {nombre_resampler}_{nombre_dataset}_train.csv para este dataset.")
            continue

        df_res = pd.DataFrame(X_res, columns=columnas)
        df_res[col_target] = y_res  # y_res ya es array, no .values

        df_res.to_csv(path_salida, index=False)
        print(f"   ✅ Guardado: {path_salida}")


### 🧬 Aumento de Datasets mediante Técnicas de Sobremuestreo

En esta etapa se genera una versión balanceada de cada dataset original mediante la aplicación de técnicas de sobremuestreo, con el objetivo de mitigar el desbalance de clases antes del entrenamiento de los modelos.

Actualmente, se emplea la técnica:

- `PCSMOTE` (Percentile-Controlled SMOTE), que permite controlar la generación de muestras sintéticas en función de percentiles de densidad, riesgo y pureza.

Para cada dataset, se exploran combinaciones específicas de parámetros según la técnica utilizada. Los datasets resultantes se almacenan en el directorio `datasets/datasets_aumentados/`, utilizando nombres de archivo que reflejan la configuración empleada (por ejemplo: `pcsmote_nombre_D25_R50_Pentropia_train.csv`).

> ⚠️ Esta fase no incluye entrenamiento ni validación de modelos. Su único propósito es generar conjuntos de datos aumentados a partir del conjunto de entrenamiento. La partición `train/test` se realiza previamente, y **solo la parte de entrenamiento es sometida a sobremuestreo**. El conjunto de prueba permanece sin modificar para garantizar una evaluación imparcial posterior.


In [6]:
# --- GRID PCSMOTE ---
percentiles_densidad = [25, 50, 75]
percentiles_riesgo = [60]#[25, 50, 75]
criterios_pureza = ["entropia", "proporcion"]

# acá se definen TODAS las variantes de IsolationForest que se quieren comparar
percentiles_isolation = [0] #[0, 1]  # ej.: 0% y 1%; aca se puede agregar 5, etc.

combinaciones = list(product(
    percentiles_densidad,
    percentiles_riesgo,
    criterios_pureza,
))

# --- Rutas base ---
os.makedirs("../logs/", exist_ok=True)

RUTA_CLASICOS = "../datasets/datasets_aumentados/resampler_clasicos/"
os.makedirs(RUTA_CLASICOS, exist_ok=True)

datasets_a_ignorar = {
    "eurosat",
    "shuttle",
    "glass",
    "heart",
    "wdbc",
    # "ecoli",
    "iris",
    "us_crime"
}

for nombre_dataset, config in config_datasets.items():
    if nombre_dataset in datasets_a_ignorar:
        continue

    print(f"\n📁 Dataset: {nombre_dataset}")

    # ───────────────── Reset del acumulador de logs POR MUESTRA ─────────────────
    # Se hace UNA sola vez por dataset para acumular:
    #   - todas las combinaciones D/R/P
    #   - y todos los percentiles de isolation (I0, I1, etc.)
    if nombre_dataset in PCSMOTE._acumulador_logs_por_muestra_por_dataset:
        del PCSMOTE._acumulador_logs_por_muestra_por_dataset[nombre_dataset]

    # ───────────────── Bucle sobre LOS DIFERENTES percentiles de Isolation ─────────────────
    for percentil_isolation_cleaner in percentiles_isolation:
        print(f"\n   ▶ Ejecutando variante con IsolationForest = {percentil_isolation_cleaner}%")

        # --- CASO BASE (única lectura del dataset crudo por variante de isolation) ---
        base_train, base_test = generar_caso_base(
            nombre_dataset,
            config,
            porcentaje_limpieza=percentil_isolation_cleaner,
        )
        print(f"🟦 Caso base generado (I{percentil_isolation_cleaner}):\n - Train: {base_train}\n - Test: {base_test}")

        # Cargar en memoria el TRAIN base ya escalado (y limpiado si I>0)
        col_target = config.get("col_target", "target")
        df_base_train = pd.read_csv(base_train)

        if col_target not in df_base_train.columns:
            raise ValueError(f"La columna target '{col_target}' no está en {base_train}")

        # Versión DataFrame/Series para los clásicos
        X_train_df = df_base_train.drop(columns=[col_target])
        y_train_series = df_base_train[col_target]

        # Versión numpy para PCSMOTE
        X_train_base = X_train_df.values
        y_train_base = y_train_series.values

        # Para nombrar archivos (sufijo I0, I1, etc.)
        percentil_isolation_etiqueta = percentil_isolation_cleaner

        # --- AUMENTACIÓN CLÁSICA (SMOTE, BorderlineSMOTE, ADASYN) ---
        generar_aumentaciones_clasicas_y_guardar(
            nombre_dataset=nombre_dataset,
            X_train=X_train_df,
            y_train=y_train_series,
            col_target=col_target,
            ruta_clasicos=RUTA_CLASICOS,
            overwrite=False,
            # solo se usa para el nombre de archivo: smote_dataset_I0_train.csv, etc.
            percentil_isolation_etiqueta=percentil_isolation_etiqueta,
        )

        # --- GRID DE PCSMOTE + ISOLATION CLEANER SOBRE ESTE TRAIN BASE ---        
        for idx, (pdens, priesgo, criterio) in enumerate(combinaciones, start=1):

            print(
                f"#{idx:02d} ➕ Aumentando con "
                f"D={pdens} | R={priesgo} | P={criterio} | I={percentil_isolation_cleaner}"
            )

            path_train, sampler = aumentar_dataset_pcsmote_y_guardar(
                nombre_dataset=nombre_dataset,
                X_train_base=X_train_base,
                y_train_base=y_train_base,
                percentil_densidad=pdens,
                percentil_riesgo=priesgo,
                criterio_pureza=criterio,
                # solo para nombre del archivo pcsmote_{dataset}_..._I*_train.csv
                percentil_isolation_etiqueta=percentil_isolation_etiqueta,
                col_target=col_target,
            )

            if path_train is None or sampler is None:
                print("❌ Falló la generación.")
                continue

            sampler.nombre_dataset = nombre_dataset

            nombre_configuracion = (
                f"D{int(pdens)}_R{int(priesgo)}"
                f"_{criterio}_I{percentil_isolation_cleaner}"
            )
            sampler.nombre_configuracion = nombre_configuracion

            print(f"✅ Guardado exitoso:\n - Train: {path_train}")

            # --- rutas de logs (sanitizadas) ---
            base_logs = Path("../datasets/datasets_aumentados/logs/pcsmote")
            p_tag = Utils.tag_p(criterio)  # "Pentropia"/"Pproporcion" 

            # Log por clase: se diferencia POR configuración, incluyendo I
            fname_clase = Utils.safe_token(
                f"log_pcsmote_{nombre_dataset}_D{pdens}_R{priesgo}_{p_tag}_I{percentil_isolation_cleaner}.csv"
            )

            # Log por muestra: UN SOLO EXCEL POR DATASET
            # Dentro del Excel se distinguen:
            #   - configuraciones D/R/P/I por columna 'configuracion'
            fname_muestra = Utils.safe_token(
                f"log_pcsmote_x_muestra_{nombre_dataset}.xlsx"
            )

            log_path_x_clase = str(base_logs / "por_clase" / fname_clase)
            log_path_x_muestras = str(base_logs / "por_muestras" / fname_muestra)

            # Asegurar subdirectorios de logs
            os.makedirs(base_logs / "por_clase", exist_ok=True)
            os.makedirs(base_logs / "por_muestras", exist_ok=True)

            # --- export logs (CSV/Excel) ---
            sampler.exportar_log_csv(log_path_x_clase)
            sampler.exportar_log_muestras_excel(log_path_x_muestras)

            print(f"📄 Log exportado: {log_path_x_clase}")
            print(f"📄 Log exportado: {log_path_x_muestras}")



📁 Dataset: ecoli

   ▶ Ejecutando variante con IsolationForest = 0%
⚠️ Caso base ya existe para ecoli. Usando archivos existentes.
🟦 Caso base generado (I0):
 - Train: ../datasets/datasets_aumentados/base/ecoli_I0_train.csv
 - Test: ../datasets/datasets_aumentados/base/ecoli_test.csv
🔧 Aumentación clásica (en memoria) para: ecoli
   X_train shape: (268, 7), y_train shape: (268,)
   🔁 Aplicando smote ...
   ⚠️ smote no generó muestras sintéticas: Expected n_neighbors <= n_samples_fit, but n_neighbors = 6, n_samples_fit = 2, n_samples = 2
      → Se omite guardar smote_ecoli_train.csv para este dataset.
   🔁 Aplicando borderlinesmote ...
   ⚠️ borderlinesmote no generó muestras sintéticas: Expected n_neighbors <= n_samples_fit, but n_neighbors = 6, n_samples_fit = 4, n_samples = 3
      → Se omite guardar borderlinesmote_ecoli_train.csv para este dataset.
   🔁 Aplicando adasyn ...
   ⚠️ adasyn no generó muestras sintéticas: Expected n_neighbors <= n_samples_fit, but n_neighbors = 6, n_s

c:\Users\FamiliaNatelloMedina\Documents\UNLu\armado-tesina\codigo\notebooks\../scripts\pc_smote.py:173: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(lista, ignore_index=True)


📄 Log exportado: ..\datasets\datasets_aumentados\logs\pcsmote\por_clase\log_pcsmote_ecoli_D25_R60_Pproporcion_I0.csv
📄 Log exportado: ..\datasets\datasets_aumentados\logs\pcsmote\por_muestras\log_pcsmote_x_muestra_ecoli.xlsx
#03 ➕ Aumentando con D=50 | R=60 | P=entropia | I=0
self.riesgo recibido en el construct:  60.0
self.riesgo recibido en el construct:  60.0
Umbral por percentil riesgo: 0.2857142857142857
self_meta: {'umbral_densidad': 0.0, 'umbral_entropia': 0.5916727785823274, 'riesgo_medio': 0.49553571428571425, 'riesgo_std': 0.1987986078411814, 'densidad_media': 0.3397345823575332, 'vecinos_validos_promedio': 1.4098360655737705, 'n_candidatas': 61, 'n_filtradas': 32, 'elapsed_ms': None, 'k_efectivo': 7, 'umbral_riesgo_min': 0.2857142857142857, 'percentil_densidad_distancias_elegido': 20.0, 'valor_percentil_global_elegido': 0.4659729074200057, 'umbral_densidad_global': 0.4659729074200057, 'k_global': 7, 'cant_vecinos_en_p_elegido': 1.4098360655737705, 'cant_min_en_p_elegido': 0.

c:\Users\FamiliaNatelloMedina\Documents\UNLu\armado-tesina\codigo\notebooks\../scripts\pc_smote.py:173: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(lista, ignore_index=True)


📄 Log exportado: ..\datasets\datasets_aumentados\logs\pcsmote\por_clase\log_pcsmote_ecoli_D50_R60_Pentropia_I0.csv
📄 Log exportado: ..\datasets\datasets_aumentados\logs\pcsmote\por_muestras\log_pcsmote_x_muestra_ecoli.xlsx
#04 ➕ Aumentando con D=50 | R=60 | P=proporcion | I=0
self.riesgo recibido en el construct:  60.0
self.riesgo recibido en el construct:  60.0
Umbral por percentil riesgo: 0.2857142857142857
self_meta: {'umbral_densidad': 0.0, 'umbral_entropia': None, 'riesgo_medio': 0.49553571428571425, 'riesgo_std': 0.1987986078411814, 'densidad_media': 0.3397345823575332, 'vecinos_validos_promedio': 1.4098360655737705, 'n_candidatas': 61, 'n_filtradas': 32, 'elapsed_ms': None, 'k_efectivo': 7, 'umbral_riesgo_min': 0.2857142857142857, 'percentil_densidad_distancias_elegido': 20.0, 'valor_percentil_global_elegido': 0.4659729074200057, 'umbral_densidad_global': 0.4659729074200057, 'k_global': 7, 'cant_vecinos_en_p_elegido': 1.4098360655737705, 'cant_min_en_p_elegido': 0.88524590163934

c:\Users\FamiliaNatelloMedina\Documents\UNLu\armado-tesina\codigo\notebooks\../scripts\pc_smote.py:173: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(lista, ignore_index=True)


📄 Log exportado: ..\datasets\datasets_aumentados\logs\pcsmote\por_clase\log_pcsmote_ecoli_D50_R60_Pproporcion_I0.csv
📄 Log exportado: ..\datasets\datasets_aumentados\logs\pcsmote\por_muestras\log_pcsmote_x_muestra_ecoli.xlsx
#05 ➕ Aumentando con D=75 | R=60 | P=entropia | I=0
self.riesgo recibido en el construct:  60.0
self.riesgo recibido en el construct:  60.0
Umbral por percentil riesgo: 0.2857142857142857
self.riesgo recibido en el construct:  60.0
self.riesgo recibido en el construct:  60.0
self.riesgo recibido en el construct:  60.0
Umbral por percentil riesgo: 0.5714285714285714
self.riesgo recibido en el construct:  60.0
Umbral por percentil riesgo: 0.2857142857142857
self.riesgo recibido en el construct:  60.0
self.riesgo recibido en el construct:  60.0
Umbral por percentil riesgo: 0.2857142857142857
✅ Guardado exitoso:
 - Train: ..\datasets\datasets_aumentados\pcsmote_ecoli_D75_R60_Pentropia_I0_train.csv
📁 Log por clase guardado en: ..\datasets\datasets_aumentados\logs\pcsmot